In [37]:
import pandas as pd
import plotly.express as px
import statsmodels

pd.options.plotting.backend = "plotly"

def read_in_realtime():
    return pd.read_json('./data/realtime.json')

def fix_realtime(df):
    df['int_tolid'] = df['sample_id'].astype(str).str.split('_',n=1).str[0]
    return df

def read_in_informatics():
    informatics = pd.read_csv('./data/informatics.tsv', delimiter='\t')
    informatics.drop(columns=['statussummary','status','asm', 'jira', 'accession', 'organelle', 'uli', 'common name',
                        'sex', 'genomic sex', 'ploidy', 'taxon', 'family', 'genus', 'order', 'phylum', 'BioSample',
                        'BioProject', 'vgp_orders', 'vgp_plus', 'asg', 'erga', '25g', 'britain&ireland', 'dnazoo',
                        'cichlids', 'BADASS', 'rodents', 'jiggins', 'murchison', 'R&D', 'protist-microalgae', 'UKbutterflies',
                        '10XG cov (gscope)', '10XG cov (sts)', 'last_data', 'sts pacbio status (new)','sts hic status (new)',
                        'hic_from', 'PacBio','ONT','10XG','Htag','BioNano','HiC','RNAseq','IsoSeq','note or paper','raw data sub',
                        'submission todo','contig N50 (Mb)','contig count','scaffold N50 (Mb)','scaffold count','longest (Mb)',
                        'length (Mb)','BUSCO','QV','runs','notes','pipeline status','Unnamed: 0', 'coi', 'completeness',
                        'darwin', 'sts size (Mb)', 'est. size (Mb)', 'long read cov (gscope)', 'long read cov (sts)'], inplace=True)
    return informatics

def fix_informatics(informatics):
    informatics['prefix'] = informatics['sample'].astype(str).str[:2]
    informatics = informatics.rename(columns={'sample':'int_tolid'})
    return informatics.query("`prefix` == 'id' ")

def read_in_tolqc():
    return pd.read_json('./data/tolqc.json')

def fix_tolqc(df):
    new_df = df[['specimen','N50','N90',"plot-ccs_readlength_hist", 'group']]
    new_df['project'] = new_df['group'].astype(str).str.split('/').str[0]
    new_df['prefix'] = new_df['specimen'].astype(str).str[:2]
    new_df = new_df.rename(columns={'sample':'int_tolid'})
    return new_df.query("`prefix` == 'id' & `project` == 'darwin'")

def subset_tolqc(realtime, tolqc):
    return tolqc[tolqc['specimen'].isin(realtime['int_tolid'].tolist())]

def merge_frames(df1, df2):
    merged = pd.merge(df1,
                df2[['int_tolid','htzgy (%)', 'rep frac (%)']],
                on='int_tolid',
                how='left')
    return merged

def graph_hetero(df):
    df.sort_values('htzgy (%)',inplace=True)
    fig = px.scatter(x=df['length_after'],
                    y=df['htzgy (%)'],
                    title='Length after curation by Heterozygosity (%)',
                    color=df['sample_id'])
    fig.show()

def graph_length_change(df):
    new = df.query("`sample_id` != 'idPheCory1_1.0' and `sample_id` != 'idNowFero1_1.0'")
    fig = px.scatter(x=new['sample_id'],
                    y=new['length_change'],
                    title='Sample_id by Change in length (%)',
                    color=new['sample_id'])
    fig.show()
    print(f'OUTLIERS = graph_length_change: idPheCory1_1.0 = -5.5 | idNowFero1_1.0 = +2.4')

def graph_rep(df):
    df.sort_values('rep frac (%)',inplace=True)
    fig = px.scatter(x=df['length_after'],
                    y=df['rep frac (%)'],
                    title='Length (post-curation) by Repeat Fraction (%)',
                    color=df['sample_id'],
                    trendline="ols") # Clearly a trend here, but line doesn't work
    fig.show()
    print(f'')

def graph_rep_change(df):
    df.sort_values('rep frac (%)',inplace=True)
    fig = px.scatter(x=df['rep frac (%)'],
                    y=df['length_change'],
                    title='Length change(%) by Repeat Fraction (%)',
                    color=df['sample_id'],
                    trendline="ols") # Clearly a trend here, but line doesn't work
    fig.show()

def graph_length_change(df):
    fig = px.scatter(x=df['sample_id'],
                    y=(df['scaff_count_after']-df['scaff_count_before'])/df['scaff_count_before']*100,
                    title='Sample_id by Change in scaffold count (%)',
                    color=df['sample_id'])
    fig.show()
    print(f'')

def graph_n50(df):
    fig = px.scatter(x=df['specimen'],
                y=df['N50'],
                title='Specimen by N50 (no. of reads that contain 50% of estimated genome)',
                color=df['specimen'])
    fig.show()
    print(f'')

def graph_n90(df):
    fig = px.scatter(x=df['specimen'],
                y=df['N90'],
                title='Specimen by N90 (no. of reads that contain 90% of estimated genome)',
                color=df['specimen'])
    fig.show()
    print(f'')

def main():
    realtime = read_in_realtime()
    fixed_realtime = fix_realtime(realtime)

    informatics = read_in_informatics()
    fixed_informatics = fix_informatics(informatics)

    tolqc = read_in_tolqc()
    fixed_tolqc = fix_tolqc(tolqc)
    s_tolqc = subset_tolqc(fixed_realtime, fixed_tolqc)

    merged = merge_frames(fixed_realtime, fixed_informatics)
    graph_hetero(merged)
    graph_length_change(merged)
    graph_rep(merged)
    graph_rep_change(merged)
    graph_length_change(merged)
    graph_n50(s_tolqc)
    graph_n90(s_tolqc)

if __name__ == '__main__':
    main()

<ipython-input-37-3e4fd2172445>:37: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-37-3e4fd2172445>:38: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

